In [7]:
import pandas as pd

# load results - datacite, scholex and overton

# nerc_citations_2025/results/latest_results_dataCite.csv
# /home/matnic/Projects/citationNotebook/nerc_citations_2025/results/latest_results_dataCite.csv
dataCite_df = pd.read_csv("results/latest_results_dataCite.csv")
scholex_df = pd.read_csv("results/latest_results_scholex.csv")
overton_df = pd.read_csv("results/latest_results_overton.csv")
# merge in a general way

In [8]:
with pd.option_context('display.max_colwidth', None):
    print(dataCite_df.iloc[1])


pub_doi                                                                                                              https://doi.org/10.1002/2015gl065750
source-id                                                                                                                               datacite-crossref
relation-type-id                                                                                                                              is-cited-by
occurred-at                                                                                                                      2019-08-01T11:28:04.000Z
data_doi                                                                                                     10.5285/6feac38a-5847-46f9-84e4-e7e9d291f935
data_publisher                                                                                                                    Polar Data Centre (PDC)
data_title                                                        Snow accum

In [9]:
with pd.option_context('display.max_colwidth', None):
    print(scholex_df.iloc[205])

relationshipType                                                                                                                                                      IsReferencedBy
pub_title                                    Anthropogenic Drivers of Variation in Concentrations of Perfluoroalkyl Substances in Otters (<i>Lutra lutra</i>) from England and Wales
pub_date                                                                                                                                                                  2022-01-11
pub_doi                                                                                                                                                      10.1021/acs.est.1c05410
data_doi                                                                                                                                10.5285/a1f88807-4826-44bc-994d-a902da5119c2
data_title                                                                                     

In [11]:
with pd.option_context('display.max_colwidth', None):
    print(overton_df.iloc[51])


data_doi                                                                                  10.5285/836f016a-33be-6ddc-e053-6c86abc0788e
data_publisher                                                                                British Oceanographic Data Centre (BODC)
data_title                                             The GEBCO_2019 Grid - a continuous terrain model of the global oceans and land.
data_dates                        [{'date': '2019-01-01/2019-01-01', 'dateType': 'Collected'}, {'date': '2019', 'dateType': 'Issued'}]
data_publication_year                                                                                                           2019.0
data_authors                                                                              ['GEBCO Bathymetric Compilation Group 2019']
relation_type                                                                                                               references
pub_doi                  https://www.marineboard.eu/pub

In [ ]:
ideal_columns = ['data_doi', 'data_publisher', 'data_title', 'data_dates', 'data_publication_year', 'data_authors', 'data_registered', 'relation_type', 'pub_doi', 'pub_title', 'pub_date', 'pub_authors']


In [11]:
# take data_doi and pub_doi columns from each df, concatenate vertically
doi_df = pd.concat([scholex_doi_df[['data_doi', 'pub_doi']], dataCite_doi_df[['data_doi', 'pub_doi']]], ignore_index=True)
doi_df_unique = doi_df.drop_duplicates(subset=['data_doi', 'pub_doi'])
print(len(doi_df))
print(len(doi_df_unique))

26726
18542


In [ ]:
# if all sources are wanted, 
print(dataCite_df.columns)
print(scholex_df.columns)
print(overton_df.columns)




Index(['pub_doi', 'source-id', 'relation-type-id', 'occurred-at', 'data_doi',
       'data_publisher', 'data_title', 'data_dates', 'data_publication_year',
       'data_authors', 'data_registered'],
      dtype='object')
Index(['relationshipType', 'pub_title', 'pub_date', 'pub_doi', 'data_doi',
       'data_title', 'data_authors', 'data_publisher', 'pub_authors',
       'source_id'],
      dtype='object')
Index(['data_doi', 'data_publisher', 'data_title', 'data_dates',
       'data_publication_year', 'data_authors', 'relation_type', 'pub_doi',
       'pub_title', 'pub_date', 'pub_authors', 'source-id'],
      dtype='object')


In [28]:
print(len(dataCite_df.columns))
print(len(scholex_df.columns))
print(len(overton_df.columns))



11
10
12
